## PART 3-MIDAS_TASK_2
In this notebook I have trained a pretrained model on the dataset provided.


## Importing the libraires

In [1]:
import os
import zipfile
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Lambda, MaxPooling2D, BatchNormalization, Add
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input# core layers
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau , ModelCheckpoint

from platform import python_version

## Loading the pretrained model

In [2]:
model =  tf.keras.models.load_model('/content/best_model_pretrained.h5')
model.summary()
model

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 12, 12, 64)        256       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 10, 10, 128)       7385

## Loading the dataset


In [8]:
local_zip = '/content/mnistTask3.zip'
zip_ref   = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('tmp/imgclassifi')
zip_ref.close()

NameError: name 'zipfile' is not defined

In [4]:
train_dir = os.path.join('tmp/imgclassifi/mnistTask')
sub_dir = [os.path.join(train_dir,o) for o in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir,o))]
sub_dir.sort()
sub_dir

['tmp/imgclassifi/mnistTask/0',
 'tmp/imgclassifi/mnistTask/1',
 'tmp/imgclassifi/mnistTask/2',
 'tmp/imgclassifi/mnistTask/3',
 'tmp/imgclassifi/mnistTask/4',
 'tmp/imgclassifi/mnistTask/5',
 'tmp/imgclassifi/mnistTask/6',
 'tmp/imgclassifi/mnistTask/7',
 'tmp/imgclassifi/mnistTask/8',
 'tmp/imgclassifi/mnistTask/9']

### FEW POINTS REGARDING THE DATASET
###### The dataset has exterme noise since each subdirectories contains a mix of 0-9 images,The model will failded to achieve result

## Data preprocessing and argumentation


In [5]:
batchsize = 128
EPOUCH    = 50

train_datagen = ImageDataGenerator(  
       
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range= 10,     
        zoom_range = 0.1,  
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False,  
        vertical_flip=False,
        validation_split = 0.2)



train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(28,28),  # All images will be resized to 28*28
        batch_size=batchsize,
        color_mode = "grayscale",
        class_mode='categorical',
        subset = 'training')

validation_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(28,28),  # All images will be resized to 28*28
        batch_size=batchsize,
        color_mode = "grayscale",
        class_mode='categorical',
        subset = 'validation')

rlr = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.0001)   # minimun leraing rate 0.0001
ckp = ModelCheckpoint('best_model.h5', monitor = 'val_accuracy',verbose = 1, save_best_only = True, mode = 'max')


Found 48006 images belonging to 10 classes.
Found 11994 images belonging to 10 classes.


## Training The  pretrained model

In [6]:
history = model.fit_generator(train_generator,steps_per_epoch= 48006/batchsize,epochs = EPOUCH ,verbose=1,validation_data=validation_generator, callbacks=[rlr,ckp])




/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
375/375 [==============================] - 62s 85ms/step - loss: 2.9194 - accuracy: 0.1049 - val_loss: 2.3203 - val_accuracy: 0.1131

Epoch 00001: val_accuracy improved from -inf to 0.11314, saving model to best_model.h5
Epoch 2/50
375/375 [==============================] - 30s 80ms/step - loss: 2.4330 - accuracy: 0.1095 - val_loss: 2.2952 - val_accuracy: 0.1080

Epoch 00002: val_accuracy did not improve from 0.11314
Epoch 3/50
375/375 [==============================] - 31s 82ms/step - loss: 2.3732 - accuracy: 0.1088 - val_loss: 2.2992 - val_accuracy: 0.1125

Epoch 00003: val_accuracy did not improve from 0.11314
Epoch 4/50
375/375 [==============================] - 30s 81ms/step - loss: 2.3333 - accuracy: 0.1117 - val_loss: 2.2789 - val_accuracy: 0.1079

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0003124999930150807.

Epoch 00004: val_accuracy did not improve from 0.11314
Epoch 5/50
375/375 [==============================] - 30s 81ms/step - loss: 2.3125 - ac

## Results 
Training accuracy  = 0.122
validation accuracy = 0.1148

## Loading the Mnist dataset

preproceesing the testing data


In [18]:
from tensorflow.keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_test = X_test/255.0


In [7]:
# one hot encoding for the classes

In [20]:
from keras.utils import np_utils
n_classes = 10
Y_test = np_utils.to_categorical(Y_test, n_classes)

## Evaluating the model

In [22]:
score = model.evaluate(X_test,Y_test)

313/313 [==============================] - 1s 4ms/step - loss: 2.3536 - accuracy: 0.0684


In [ ]:
## Result 
ACCuracy = 0.0684 (As expected)

# FINAL OVERVIEW

As expected the Model Performance is Dead, we unable to achieve even 1% accuracy on the Mnist test set, As already mentioned the Datset is Highly noisy.

# Method to achieve The result

In order to achieve the good result we have to preform the Data cleansing, in order to reduce the noise(making a balanced dataset by puuting each image in its respective directory) we can perform Follwing operations,

1) Using The k-means custering alogorithms we can cluster the same type of image in the a particular subdirectory.It can achieve around the 90 to 95% of accuracy.
2) Use the previous model part 2 to label the imaage as its is highly optimized Model with the validation accuracy around 99.5,then perform the K-mean clustering on the 0.5% of the data that will be classified wrong by the Part-2 Model.

As far as My understanding is concerned The second method will be more beneficial since the specification of images in the dataset is compeletely matched with the Mnist dataset, and we can easliy label the image.